In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# Load data 
n_classes = 2

nx = 121
ny = 90
nz = 121

In [ ]:
# Create network architecture

EPOCHS = 25
BATCH_SIZE = 64

from tensorflow.contrib.layers import flatten

def BrainHackNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1

    # Layer 1: Convolutional. Input = nX x nY x nZ. Output = 28x28x6
    # Filter = [depth, height, width, channels=1, output=?.
    # Output size for X = ceil(float(in_X_dim - filter_x_dim + 1)/float(stride_X_dim)) 
    conv1_W = tf.Variable(tf.truncated_normal(shape=(10, 10, 10, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv3d(x, conv1_W, strides=[1, 5, 5, 5, 1], padding='VALID') + conv1_b

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = Output of previous layer. Output = (in-filter)/stride. (3rd dim will still be '6')
    conv1 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Output = () X 16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv3d(conv1, conv2_W, strides=[1, 2, 2, 2, 1], padding='VALID') + conv2_b

    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = same as previous o/p. Output = 5x5x16.
    conv2 = tf.nn.max_pool3d(conv2, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='VALID')

    # Flatten. Input = previous size . Output = product of previous size.
    fc0   = flatten(conv2)

    # Layer 3: Fully Connected. Input = previous. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(64, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b

    # Activation.
    fc1    = tf.nn.relu(fc1)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b

    # Activation.
    fc2    = tf.nn.relu(fc2)

    # Layer 5: Fully Connected. Input = 84. Output = nclasses = 2.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(n_classes))
    logits = tf.matmul(fc2, fc3_W) + fc3_b

    return logits


In [ ]:
# One hot encode the labels and define placeholders for inputs
# Input = [batch, d, h, w, channels]
x = tf.placeholder(tf.float32, (None, nz, ny, nz, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
### Train your model here.
rate = 0.001

logits = BrainHackNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated.
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
# The big training loop

from sklearn.utils import shuffle

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        #print("EPOCH {} ...".format(i+1))
        print("EPOCH {} Valid Acc = {:.3f}".format(i+1,validation_accuracy))
    
    print()
        
    saver.save(sess, './lenet_traffic')
    print("Model saved")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    training_accuracy = evaluate(X_train, y_train)

print("Training Accuracy = {:.3f}".format(training_accuracy))
print()

In [ ]:
prediction = tf.nn.softmax(logits)
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    new_predictions = sess.run(prediction, feed_dict={x: new_input})     

In [ ]:
with tf.Session() as sess:
    print(sess.run(tf.nn.top_k(tf.constant(new_predictions), k=5)))